In [1]:
%%sh
pip install Pygments -q
echo "Please restart your runtime now (Runtime -> Restart runtime). This ensures that the correct dependencies are loaded."

Please restart your runtime now (Runtime -> Restart runtime). This ensures that the correct dependencies are loaded.


# Based on these tutorials:

- https://docs.feast.dev/reference/dqm
- https://docs.feast.dev/tutorials/validating-historical-features


In [1]:
!rm -rf dqm_repo
!feast init dqm_repo


Creating a new Feast repository in /home/francisco/github/feast/examples/data-quality-monitoring/dqm_repo.



In [2]:
%cd dqm_repo
!mkdir logs
!ls -R

/home/francisco/github/feast/examples/data-quality-monitoring/dqm_repo
.:
data  example.py  feature_store.yaml  __init__.py  logs

./data:
driver_stats.parquet

./logs:


In [3]:
!pygmentize feature_store.yaml

project: dqm_repo
registry: data/registry.db
provider: local
online_store:
    path: data/online_store.db
entity_key_serialization_version: 2


In [4]:
import pandas as pd

pd.read_parquet("data/driver_stats.parquet")

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2022-07-28 05:00:00+00:00,1005,0.840626,0.487525,754,2022-08-12 05:47:40.134
1,2022-07-28 06:00:00+00:00,1005,0.942585,0.330828,580,2022-08-12 05:47:40.134
2,2022-07-28 07:00:00+00:00,1005,0.922485,0.177851,830,2022-08-12 05:47:40.134
3,2022-07-28 08:00:00+00:00,1005,0.839960,0.033974,120,2022-08-12 05:47:40.134
4,2022-07-28 09:00:00+00:00,1005,0.337953,0.362264,252,2022-08-12 05:47:40.134
...,...,...,...,...,...,...
1802,2022-08-12 03:00:00+00:00,1001,0.550637,0.392341,158,2022-08-12 05:47:40.134
1803,2022-08-12 04:00:00+00:00,1001,0.979974,0.097716,758,2022-08-12 05:47:40.134
1804,2021-04-12 07:00:00+00:00,1001,0.269647,0.906581,74,2022-08-12 05:47:40.134
1805,2022-08-04 17:00:00+00:00,1003,0.492565,0.314931,402,2022-08-12 05:47:40.134


In [5]:
!pygmentize -f terminal16m example.py

# This is an example feature definition file

from datetime import timedelta

from feast import Entity, FeatureService, FeatureView, Field, FileSource
from feast.types import Float32, Int64

# Read data from parquet files. Parquet is convenient for local development mode. For
# production, you can use your favorite DWH, such as BigQuery. See Feast documentation
# for more info.
driver_hourly_stats = FileSource(
    name="driver_hourly_stats_source",
    path="/home/francisco/github/feast/examples/data-quality-monitoring/dqm_repo/data/driver_stats.parquet",
    timestamp_field="event_timestamp",
    created_timestamp_column="created",
)

# Define an entity for the driver. You can think of entity as a primary key used to
# fetch features.
driver = Entity(name="driver", join_keys=["driver_id"])

# Our parquet files contain sample data that includes a driver_id column, timestamps and
# three feature column. Here we define a Feature View that will allow us to serve this
# data to our model 

In [6]:
feature_store = \
f"""project: dqm_repo
registry: data/registry.db
provider: local
offline_store:
    type: file
online_store:
    path: data/online_store.db
entity_key_serialization_version: 2
feature_server:
    enabled: True
    feature_logging:
        enabled: True
        flush_interval_secs: 60
        write_to_disk_interval_secs: 10
    
go_feature_retrieval: False
"""

with open('feature_store.yaml', "w") as feature_store_file:
    feature_store_file.write(feature_store)

dqm_services = f"""
from feast import FeatureService
from feast.feature_logging import LoggingConfig
from feast.infra.offline_stores.file_source import FileLoggingDestination

from example import driver_hourly_stats, driver, driver_hourly_stats_view

fs = FeatureService(
    name="driver_activity", 
    features=[driver_hourly_stats_view],
    logging_config=LoggingConfig(
        sample_rate=1.0,
        destination=FileLoggingDestination(path="logs/"),
    )
)
"""
    
with open('dqm_services.py', "w") as dqm_services_file:
    dqm_services_file.write(dqm_services)

In [7]:
!feast apply

Created entity driver
Created feature view driver_hourly_stats
Created feature service driver_activity

Created sqlite table dqm_repo_driver_hourly_stats



In [8]:
from datetime import datetime, timedelta
import pandas as pd

from feast import FeatureStore

# The entity dataframe is the dataframe we want to enrich with feature values
entity_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001, 1002, 1003],
        "label_driver_reported_satisfaction": [1, 5, 3], 
        "event_timestamp": [
            datetime.now() - timedelta(minutes=11),
            datetime.now() - timedelta(minutes=36),
            datetime.now() - timedelta(minutes=73),
        ],
    }
)

store = FeatureStore(repo_path=".")

training_data_job = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
)

In [9]:
training_data_job.to_df()

,driver_id,label_driver_reported_satisfaction,event_timestamp,conv_rate,acc_rate,avg_daily_trips
0,1002,5,2022-08-12 05:12:24.271472+00:00,0.138436,0.526138,295
1,1001,1,2022-08-12 05:37:24.271467+00:00,0.979974,0.097716,758
2,1003,3,2022-08-12 04:35:24.271474+00:00,0.924060,0.768807,370


In [10]:
training_data_job = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
)
training_data_job.to_df()

,driver_id,label_driver_reported_satisfaction,event_timestamp,conv_rate,acc_rate,avg_daily_trips
0,1002,5,2022-08-12 05:12:24.271472+00:00,0.138436,0.526138,295
1,1001,1,2022-08-12 05:37:24.271467+00:00,0.979974,0.097716,758
2,1003,3,2022-08-12 04:35:24.271474+00:00,0.924060,0.768807,370


In [11]:
from feast.infra.offline_stores.file_source import SavedDatasetFileStorage
reference_dataset = store.create_saved_dataset(
    from_=training_data_job,
    name="reference_dataset",
    storage=SavedDatasetFileStorage(path='data/driver_stats_reference.parquet')
)
reference_dataset

/home/francisco/github/feast/sdk/python/feast/feature_store.py:1127: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


<SavedDataset(name = reference_dataset, features = ['driver_hourly_stats:conv_rate', 'driver_hourly_stats:acc_rate', 'driver_hourly_stats:avg_daily_trips'], join_keys = ['label_driver_reported_satisfaction', 'driver_id'], storage = <feast.infra.offline_stores.file_source.SavedDatasetFileStorage object at 0x7fa30fd6f820>, full_feature_names = False, tags = {}, feature_service_name = None, _retrieval_job = <feast.infra.offline_stores.file.FileRetrievalJob object at 0x7fa30f52aa30>, min_event_timestamp = 2022-08-12 04:35:24.271474+00:00, max_event_timestamp = 2022-08-12 05:37:24.271467+00:00, created_timestamp = 2022-08-12 11:48:44.634681, last_updated_timestamp = 2022-08-12 11:48:44.634681)>

In [20]:
from feast.dqm.profilers.ge_profiler import ge_profiler
from great_expectations.dataset import PandasDataset
from great_expectations.core.expectation_suite import ExpectationSuite

@ge_profiler
def user_features_profiler(dataset: PandasDataset) -> ExpectationSuite:
    print(dataset.columns)
    dataset.expect_column_to_exist("driver_id")
    dataset.expect_column_values_to_be_between("avg_daily_trips", 0, 1000)
    dataset.expect_column_values_to_be_between("conv_rate", 0, 1)
    dataset.expect_column_values_to_be_between("acc_rate", 0, 1)
    return dataset.get_expectation_suite()

In [16]:
ds = store.get_saved_dataset('reference_dataset')

/home/francisco/github/feast/sdk/python/feast/feature_store.py:1180: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [19]:
training_data_job.to_df()

,driver_id,label_driver_reported_satisfaction,event_timestamp,conv_rate,acc_rate,avg_daily_trips
0,1002,5,2022-08-12 05:12:24.271472+00:00,0.138436,0.526138,295
1,1001,1,2022-08-12 05:37:24.271467+00:00,0.979974,0.097716,758
2,1003,3,2022-08-12 04:35:24.271474+00:00,0.924060,0.768807,370


In [18]:
ds.to_df()

,acc_rate,avg_daily_trips,event_timestamp,conv_rate,label_driver_reported_satisfaction,driver_id
0,0.768807,370,2022-08-12 04:35:24.271474+00:00,0.924060,3,1003
1,0.526138,295,2022-08-12 05:12:24.271472+00:00,0.138436,5,1002
2,0.097716,758,2022-08-12 05:37:24.271467+00:00,0.979974,1,1001


In [21]:
validation_reference = ds.as_reference(name='user_features_profiler', profiler=user_features_profiler)
_ = training_data_job.to_df(validation_reference=validation_reference)

Index(['acc_rate', 'avg_daily_trips', 'event_timestamp', 'conv_rate',
       'label_driver_reported_satisfaction', 'driver_id'],
      dtype='object')


/home/francisco/github/feast/sdk/python/feast/infra/offline_stores/offline_store.py:96: RuntimeWarning: Dataset validation is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [22]:
from datetime import datetime
!feast materialize-incremental {datetime.now().isoformat()}

Materializing 1 feature views to 2022-08-11 23:50:26-06:00 into the sqlite online store.

driver_hourly_stats from 2022-08-11 11:50:28-06:00 to 2022-08-11 23:50:26-06:00:
100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 623.43it/s]


In [23]:
print("--- Data directory ---")
!ls data

import sqlite3
import pandas as pd
con = sqlite3.connect("data/online_store.db")
print("\n--- Schema of online store ---")
print(
    pd.read_sql_query("SELECT * FROM dqm_repo_driver_hourly_stats limit 10;", con)
)
con.close()

--- Data directory ---
driver_stats.parquet		online_store.db
driver_stats_reference.parquet	registry.db

--- Schema of online store ---
                                          entity_key     feature_name  \
0  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...        conv_rate   
1  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...         acc_rate   
2  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...  avg_daily_trips   
3  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...        conv_rate   
4  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...         acc_rate   
5  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...  avg_daily_trips   
6  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...        conv_rate   
7  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...         acc_rate   
8  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...  avg_daily_trips   
9  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...        conv_rate   

            value             event_ts                  crea

In [24]:
from pprint import pprint
from feast import FeatureStore

store = FeatureStore(repo_path=".")

feature_vector = store.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
    entity_rows=[
        {"driver_id": 1004},
        {"driver_id": 1005},
    ],
).to_dict()

pprint(feature_vector)


{'acc_rate': [0.9706599712371826, 0.4944177269935608],
 'avg_daily_trips': [314, 946],
 'conv_rate': [0.27073806524276733, 0.9529669880867004],
 'driver_id': [1004, 1005]}


In [25]:
from feast import FeatureStore
feature_store = FeatureStore('.')  # Initialize the feature store

feature_service = feature_store.get_feature_service("driver_activity")
feature_vector = feature_store.get_online_features(
    features=feature_service,
    entity_rows=[
        # {join_key: entity_value}
        {"driver_id": 1004},
        {"driver_id": 1005},
    ],
).to_dict()
pprint(feature_vector)

{'acc_rate': [0.9706599712371826, 0.4944177269935608],
 'avg_daily_trips': [314, 946],
 'conv_rate': [0.27073806524276733, 0.9529669880867004],
 'driver_id': [1004, 1005]}


In [26]:
from feast.saved_dataset import ValidationReference

ref = ValidationReference(
    name='user_features_training_ref',
    dataset_name="reference_dataset",
    profiler=user_features_profiler,
)

In [27]:
store.apply(ref)

In [28]:
import pandas as pd
insert_df = pd.DataFrame({
    "driver_id": [1003],
    "conv_rate": [-1],
    "acc_rate": [2],
    "avg_daily_trips": [1500],
    "event_timestamp": [datetime.now()],
    "created": [datetime.now()],
})

store.write_to_online_store("driver_hourly_stats", insert_df)

In [29]:
end_ts = datetime.now()
start_ts = end_ts - timedelta(minutes=10)

! feast validate --feature-service driver_activity --reference user_features_training_ref {start_ts.isoformat()} {end_ts.isoformat()}

/home/francisco/github/feast/sdk/python/feast/feature_store.py:1180: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(
/home/francisco/github/feast/sdk/python/feast/feature_store.py:2360: RuntimeWarning: Logged features validation is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(
Traceback (most recent call last):
  File "/home/francisco/github/feast/venv/bin/feast", line 33, in <module>
    sys.exit(load_entry_point('feast', 'console_scripts', 'feast')())
  File "/home/francisco/github/feast/venv/lib/python3.8/site-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/home/francisco/github/feast/venv/li